In [1]:
import nltk
#nltk.download()

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
set(stopwords.words("english"))
 
example_sent = "New idea use uh something to soften up hard foods like pineapples"
 
stop_words = set(stopwords.words('english'))
 
word_tokens = word_tokenize(example_sent)
 
filtered_sentence = [w for w in word_tokens if not w in stop_words]
 
filtered_sentence = []
 
for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)
for word in word_tokens:
    print (word + " ", end = " ")
print (" ")
for word in filtered_sentence:
    print (word + " ", end = " ")

New  idea  use  uh  something  to  soften  up  hard  foods  like  pineapples   
New  idea  use  something  soften  hard  foods  pineapples  

In [214]:
import tensorflow as tf
import numpy as np

import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords


from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.datasets import imdb
from keras.models import model_from_json
#from keras.utils import np_utils
from nltk import ngrams

import pandas as pd
import numpy as np
import simplejson

# https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
tags_vocabulary = """CC CD DT EX FW IN JJ JJR JJS LS MD NN NNS NNP NNPS 
PDT POS PRP PRP$ RB RBR RBS RP SYM TO UH VB VBD VBG VBN VBP VBZ WDT WP WRB"""   

tags_vocabulary_list = tags_vocabulary.split()

#  a list, so convert the stop words to a set
#stops_set = set(stopwords.words("english"))


# TODO: Turn this collection of methods into a class instead. 

# How many max words do we think there are in a TODO item.
# We need this for correct padding of the sentences.
maxlen = 8 
_activation = 'sigmoid'
#_activation = 
_loss = 'binary_crossentropy'
_optimizer = 'adam'
_metrics = 'accuracy'


# This can also by solved by linear regression using next values. Then I can just call .predict to get teh regression value.
# _activation = 'linear' 
# _loss = 'mse'
# _optimizer = 'rmsprop'
# _metrics = 'accuracy'
# Mean squared error regression problem: https://keras.io/getting-started/sequential-model-guide/#compilation

# generates n-grams
def generate_ngrams(string, n):
    ngram_output = ngrams(string.split(), n)
    output_array = []
    for i in ngram_output:
        output_array.append(' '.join(i))
    return output_array

# This one gets the tags only without removing the stop words.
def get_tags(text):
    # Keep leeters and numbers only
    letters_only = re.sub("[^a-zA-Z0-9]", " ", text) 
    lower_case = letters_only.lower() 
    tokens = word_tokenize(lower_case)
    return [tag[1] for tag in pos_tag(tokens)]


def get_tags_string(text):
    tokens = word_tokenize(text)
    tags = [tag[1] for tag in pos_tag(tokens)]
    result = " "
    return result.join(tags)


def get_tags_without_stop_words(text, stops_set):
    # Keep leeters only
    letters_only = re.sub("[^a-zA-Z]", " ", text) 
    # Convert to lower case, split into individual words
    word_list = letters_only.lower().split()    
    # Remove stop words
    filtered_words = [w for w in word_list if not w in stops_set]

    text_to_tokenize = " "
    text_to_tokenize = text_to_tokenize.join(filtered_words)
    tokens = word_tokenize(text_to_tokenize)
    return [tag[1] for tag in pos_tag(tokens)]


def get_feature_vector(text):
    # https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
    #text_tags = get_tags_without_stop_words(text, stops_set)
    text_tags = get_tags(text)
    vector = []
    for tag in text_tags:
        vector.append(tags_vocabulary_list.index(tag) + 1)
    return vector
# In Python, searching a set is much faster than searching


def get_text_class(text, maxlen, model):
    feature_vec = get_feature_vector(text)
    predict_matrix = np.array([feature_vec])
    test = sequence.pad_sequences(predict_matrix, maxlen=maxlen)
    predict_val = model.predict_classes(test[0:1])
    return predict_val[0][0]


# Need to debug this method. It was created to print incorrect predictions in a range
# but is not working (it can not find the feaure vector)
def test_range(test, result, feature_vectors, sentences, model):
    correct_count = 0
    incorrect_count = 0
    predicted_classes = model.predict_classes(test)
    
    for index in range(len(predicted_classes)):
        if predicted_classes[index][0] == result[index]:
            correct_count = correct_count + 1
        else:
            incorrect_count = incorrect_count + 1
            print ("Looking for: ", test[index])
            sentence_index = feature_vectors.index(test[index])
            print ("Incorrect: ", sentences[sentence_index])

    return correct_count, incorrect_count


def create_cnn_todo_item_recognizer():
    max_features = len(tags_vocabulary_list) + 1
    batch_size = 10
    embedding_dims = 50
    filters = 128
    kernel_size = 2
    hidden_dims = 250
    epochs = 15
    # play around wiht epochs, kernel size max 4

    df = pd.read_csv('/home/t-judai/ideas_dataset.csv')
    todo_items_sentences = df['Sentence'].tolist()

    # We want to train on the tags since we want to learn on the syntax.
    x = []
    for sentence in todo_items_sentences:
        x.append(get_feature_vector(sentence)) 
    y = [1] * len(x)

    # Used to analize incorrect predictions
    sentences = todo_items_sentences

    # Now let's load some examples of what is not a todo Item (South park dialogs downloaded
    # from Kaggle).
    df = pd.read_csv('/home/t-judai/seinfeld-scripts.csv')
    
    # Cleaning a bit the string
    #lines = [line.replace('\n','') for line in df['Dialogue']]
    lines = df['Dialogue']
    items_to_add = len(x)
    for line in lines:
        line = str(line).strip()
        if line.find(' ') > 3: # Poor man's way to know there are at least 4-5 words on the string
            if line.find('.') > 0:
                line = line[0:line.find('.')]
            #line = line[0:3]
            try:
                x.append(get_feature_vector(line))
                sentences.append(line)
                items_to_add = items_to_add - 1
                y.append(0)
            except:
                print ('Ignoring this line:', line)
                continue
        if items_to_add == 0:
            break

    # Used to analize incorrect predictions
    feature_vectors = x
    # At this point we have the feature vectors, and the matching sentence to do look ups.

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=33)

    print(len(x_train), 'train sequences')
    print(len(x_test), 'test sequences')

    print('Pad sequences (samples x time)')
    x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
    x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

    print('x_train shape:', x_train.shape)
    print('x_test shape:', x_test.shape)


    # Based on https://github.com/fchollet/keras/blob/master/examples/imdb_cnn.py
    # https://github.com/fchollet/keras/blob/master/examples/pretrained_word_embeddings.py

    model = Sequential()

    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    model.add(Embedding(max_features,
                        embedding_dims,
                        input_length=maxlen))

    # we add a Convolution1D, which will learn filters
    # word group filters of size filter_length:
    model.add(Conv1D(filters,
                     kernel_size,
                     padding='valid',
                     activation='relu',
                     strides=1))

    # we use max pooling:
    model.add(GlobalMaxPooling1D())


    ######
    # We add a vanilla hidden layer:
    model.add(Dense(hidden_dims)) #hidden_dims
    #model.add(Dropout(0.2))
    model.add(Activation('relu'))

    # We add a vanilla hidden layer:
    model.add(Dense(128)) #hidden_dims
    #model.add(Dropout(0.2))
    model.add(Activation('relu'))

    # We add a vanilla hidden layer:
    model.add(Dense(128)) #hidden_dims
    #model.add(Dropout(0.2))
    model.add(Activation('relu'))
    
    # We add a vanilla hidden layer:
    model.add(Dense(64))
    #model.add(Dropout(0.2))
    model.add(Activation('relu'))

    # We add a vanilla hidden layer:
    model.add(Dense(32))
    #model.add(Dropout(0.2))
    model.add(Activation('relu'))

    model.add(Dense(16))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    #####


    # We project onto a single unit output layer, and squash it with a sigmoid:
    model.add(Dense(1))
    model.add(Activation(_activation)) 

    model.compile(loss = _loss,
                  optimizer = _optimizer,
                  metrics = [_metrics])

    # Fit model on training data.
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test))


    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score', score[0])
    print('Test accuracy', score[1])


    #  serialize model to JSON
    model_json = model.to_json()
    with open("/home/t-judai/model_hack.json", "w") as json_file:
        json_file.write(simplejson.dumps(simplejson.loads(model_json), indent=4))


    # serialize weights to HDF5
    model.save_weights("/home/t-judai/model_hack_weights.h5")


    # To predict something I can do (returns an array):
    predicted_classes = model.predict_classes(x_test[10:20])


    #test_string = """Schedule the meeting with the Doctor"""
    #feature_vec = get_feature_vector(test_string)
    #predict_matrix = np.array([feature_vec])
    #test = sequence.pad_sequences(predict_matrix, maxlen=maxlen)
    #predict_val = model.predict_classes(test[0:1])
    #print(predict_val)


def load_cnn_todo_reco_model_from_path(model_path, weights_path):
    print("Loading the model from disk...", model_path)
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    # load weights into new model
    print("Loading the weights from disk...", weights_path)
    loaded_model.load_weights(weights_path, by_name=True)

    print("Compiling model...")
    loaded_model.compile(loss=_loss, optimizer=_optimizer, metrics=[_metrics])
    return loaded_model

def is_sentence_a_todo_item(test_string, model):
    feature_vec = get_feature_vector(test_string)
    predict_matrix = np.array([feature_vec])
    test = sequence.pad_sequences(predict_matrix, maxlen=maxlen)
    predict_val = model.predict(test[0:1])
    #print (model.predict(test[0:1]))
    return predict_val[0][0]


#print ("Creating the bag of words...\n")
#from sklearn.feature_extraction.text import CountVectorizer
# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.
# I can probably use the preprocessor to do the taging transformations?  
# We can try using the tfidvectorizer that has the advantage of emphasizing 
# the most important words for a given document. 
# https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words
# https://stackoverflow.com/questions/20132070/using-sklearns-tfidfvectorizer-transform
#vectorizer = CountVectorizer(analyzer = "word",   \
#                             tokenizer = None,    \
#                             preprocessor = None, \
#                             stop_words = None,   \
#                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
#train_data_features = vectorizer.fit_transform(training_tags)

# Numpy arrays are easy to work with, so convert the result to an 
# array
#train_data_features = train_data_features.toarray()


#print ("Creating the bag of words...\n")

#from sklearn.feature_extraction.text import CountVectorizer
# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.
# I can probably use the preprocessor to do the taging transformations?  
# We can try using the tfidvectorizer that has the advantage of emphasizing 
# the most important words for a given document. 
# https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words
# https://stackoverflow.com/questions/20132070/using-sklearns-tfidfvectorizer-transform
#vectorizer = CountVectorizer(analyzer = "word",   \
#                             tokenizer = None,    \
#                             preprocessor = None, \
#                             stop_words = None,   \
#                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
#train_data_features = vectorizer.fit_transform(training_tags)

# Numpy arrays are easy to work with, so convert the result to an 
# array
#train_data_features = train_data_features.toarray()


In [339]:
create_cnn_todo_item_recognizer()

Ignoring this line: (Still going along with the practical joke) You know, I was wondering
Ignoring this line: (Jokingly hits the doctor) Whose tonsils grow back? (Laughs)
Ignoring this line: Guess whose birthday's comin' up soon?
Ignoring this line: Elaine, you know, I was watching you tonight, and I realized something
Ignoring this line: Whose pen?
Ignoring this line: 'cos the other night , you know, I was sleeping with Marion I rolled over and I cut her ankle with my big toe
8568 train sequences
952 test sequences
Pad sequences (samples x time)
x_train shape: (8568, 8)
x_test shape: (952, 8)
Train on 8568 samples, validate on 952 samples
Epoch 1/15
8568/8568 [==============================] - 7s 844us/step - loss: 0.4103 - acc: 0.8327 - val_loss: 0.3593 - val_acc: 0.8550
Epoch 2/15
8568/8568 [==============================] - 6s 680us/step - loss: 0.3415 - acc: 0.8606 - val_loss: 0.3185 - val_acc: 0.8792
Epoch 3/15
8568/8568 [==============================] - 6s 683us/step - loss: 0.

In [340]:
model_path = "/home/t-judai/model_hack.json"
weights_path = "/home/t-judai/model_hack_weights.h5"
model = load_cnn_todo_reco_model_from_path(model_path, weights_path)

Loading the model from disk... /home/t-judai/model_hack.json
Loading the weights from disk... /home/t-judai/model_hack_weights.h5
Compiling model...


In [185]:
test_string = "talk to humans"
result = is_sentence_a_todo_item(test_string, model)
print (result)

1


In [186]:
import json
import pandas as pd
from urllib.request import Request, urlopen
from pandas.io.json import json_normalize
import validators

import os
#os.chdir('C:\\Users\\ggarcia\\Desktop\\ClusteringTimeline')

#from html_analyzer import GetPageContentText

import re
from nltk.corpus import stopwords # Import the stop word list
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
import string


# In Python, searching a set is much faster than searching
#  a list, so convert the stop words to a set
stops = set(stopwords.words("english"))
exclude = set(string.punctuation) 

lemma = WordNetLemmatizer()

# https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
allowed_tags_dict = {'NN': True, 'NNS': True, 'JJR': True, 'JJS': True, 
                     'JJ': True, 'VB': True}

only_nouns_tags_dict = {'NN': True, 'NNS': True}

document = "I have a new idea for a metallic dog that has robotic legs so it can roam around on its own. \
            The dog would be able to walk on planets, and since it is pretty small, \
            it can get into places that humans normally could not."
document1 = "I have a new idea for a metallic dog that can talk to humans."
document2 = "I have a new idea for rainbows and butterflies eating chocolate on the beach and watching the sunset at night"
def clean_document(document):
    # Remove non-letters
    letters_only = re.sub("[^a-zA-Z]", " ", document) 
    # Remove stop words
    stop_free = " ".join([i for i in letters_only.lower().split() if i not in stops])
    # Remove punctionation
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    # Nomalize the words to the base form
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    
    # Drop every word with a tag not included in our allowed tags dictionary
    meaningful_words_list = []
    tags = pos_tag(normalized.split())
    for tag in tags:
        #We drop long words  (most of the time they are garbage from the html)
        if tag[1] in allowed_tags_dict and len(tag[0]) < 10:
            if tag[1] in only_nouns_tags_dict and len(tag[0]) < 10:
                meaningful_words_list.append(tag[0])
    return meaningful_words_list  


#documents = get_document_collection(activities_groups)
doc = clean_document(document2)
#docs_clean = [clean_document(doc) for doc in documents]    



In [187]:
print (doc)

['idea', 'rainbow', 'butterfly', 'chocolate', 'beach', 'sunset', 'night']


In [188]:
new_sentence = ""
for word in doc:
    new_sentence += word + " "
print (new_sentence)

idea rainbow butterfly chocolate beach sunset night 


In [189]:
print (is_sentence_a_todo_item(new_sentence, model))

1


In [368]:
from nltk.corpus import words
from nltk import ngrams
sentence = "jump rope that can change lengths depending on the setting"
n = 2


#sentence = clean_document(sentence)
#sentence = " ".join(sentence)
highest_prob = []

def get_summary(sentence):
    final_sentence = ""
    sixgrams = ngrams(sentence.split(), 2) 
    total_scores = []
    total_strings = []
    for grams in sixgrams:
        string = " ".join(grams)
        score = is_sentence_a_todo_item(string, model)
        total_scores.append(score)
        total_strings.append(string)
        #print (string + " " + str(score))

    #print (len(total_scores))

    i = 1
    while i < len(total_scores):
        if total_scores[i]> 0.5  and total_scores[i-1] < 0.5:
            final_sentence += total_strings[i] + " "
            i += 2
        elif total_scores[i] < 0.5 and total_scores[i-1] > 0.5:
            final_sentence += total_strings[i-1] + " "
            i += 2
        elif total_scores[i] > 0.5 and total_scores[i-1] > 0.5:
            if total_scores[i] > total_scores[i-1]:
                final_sentence += total_strings[i] + " "
                i += 2
            else:
                final_sentence += total_strings[i-1] + " "
                i += 2
        else:
            i += 1

    list_words = final_sentence.split()
    i = 1
    while i <  len(list_words):
        if list_words[i] == list_words[i-1]:
            list_words.pop(i)
            i -= 1
        i += 1
    connected_sentence = " ".join(list_words)
    #print ("")
    #print (sentence)
    return connected_sentence

In [369]:
def get_max_length_summary(sentence):
    words = sentence.split()
    if len(words) <= 10:
        return sentence
    else:
        result = get_summary(sentence)
        words_result = result.split() 
        if len(words_result) > 10:
            while len(words_result) > 10:
                result = get_summary(result)
                words_result = result.split()
        return result
sentence = "a toy that changes shape depending on what the child is thinking of playing with"
print ("")
print (sentence)
print (get_summary(sentence))
print (get_max_length_summary(sentence))


a toy that changes shape depending on what the child is thinking of playing with
a toy that changes depending on the child playing with
a toy that changes depending on the child playing with


In [365]:
from nltk.corpus import words
if "uh" in words.words():
    print ("True")

[nltk_data] Downloading package words to /home/t-judai/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True